# Trajectory Recommendation - MEMM

In [ ]:
% matplotlib inline

import os, sys, time, pickle
import math, random
import pandas as pd
import numpy as np
import heapq as hq
from scipy.optimize import minimize
from scipy.misc import logsumexp

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler, StandardScaler, MaxAbsScaler

from joblib import Parallel, delayed
import cython

```dat_ix``` is required in notebook ```shared.ipynb```.

In [ ]:
dat_ix = 0

Run notebook ```shared.ipynb```.

In [ ]:
%run 'shared.ipynb'

Hyperparameters.

In [ ]:
N_JOBS = 6         # number of parallel jobs
MIN_MAX_SCALE = True  # feature scaling, True: MinMaxScaler, False: StandardScaler
LOGIT_C = 1
run_memm1m = True

Inference using **the List Viterbi algorithm**, which *sequentially* find the (k+1)-th best path/walk given the 1st, 2nd, ..., k-th best paths/walks.

Reference papers:
- [*Sequentially finding the N-Best List in Hidden Markov Models*](http://www.eng.biu.ac.il/~goldbej/papers/ijcai01.pdf), Dennis Nilsson and Jacob Goldberger, IJCAI 2001.
- [*A tutorial on hidden Markov models and selected applications in speech recognition*](http://www.cs.ubc.ca/~murphyk/Bayes/rabiner.pdf), L.R. Rabiner, Proceedings of the IEEE, 1989.

Implementation is adapted from the above references.

In [ ]:
class HeapItem:  # an item in heapq (min-heap)
    def __init__(self, priority, task):
        self.priority = priority
        self.task = task
        self.string = str(priority) + ': ' + str(task)
        
    def __lt__(self, other):
        return self.priority < other.priority
    
    def __repr__(self):
        return self.string
    
    def __str__(self):
        return self.string

In [ ]:
def do_inference_listViterbi(ps, L, N, unary_params, pw_params, unary_features, pw_features, y_true=None, debug=False):
    assert(L > 1)
    assert(N >= L)
    assert(ps >= 0)
    assert(ps < N)
    
    Cu = np.zeros(N, dtype=np.float)      # unary_param[p] x unary_features[p]
    Cp = np.zeros((N, N), dtype=np.float) # pw_param[pi, pj] x pw_features[pi, pj]
    
    # a intermediate POI should NOT be the start POI, NO self-loops
    for pi in range(N):
        Cu[pi] = np.dot(unary_params[pi, :], unary_features[pi, :]) # if pi != ps else -np.inf
        for pj in range(N):
            Cp[pi, pj] = -np.inf if (pj == ps or pi == pj) else np.dot(pw_params[pi, pj, :], pw_features[pi, pj, :])
            
    # forward-backward procedure: adapted from the Rabiner paper
    Alpha = np.zeros((L, N), dtype=np.float)  # alpha_t(p_i)
    Beta  = np.zeros((L, N), dtype=np.float)  # beta_t(p_i)
    for t in range(1, L):
        for pj in range(N):
            Alpha[t, pj] = np.max([Alpha[t-1, pi] + Cp[pi, pj] + Cu[pj] for pi in range(N)])
        t1 = L-t
        for pi in range(N):
            Beta[t1-1, pi] = np.max([Cp[pi, pj] + Cu[pj] + Beta[t1, pj] for pj in range(N)])
            
    Fu = np.zeros((L, N), dtype=np.float)       # f_t(p)
    Fp = np.zeros((L-1, N, N), dtype=np.float)  # f_{t, t+1}(p, p')
    for t in range(L):
        for p in range(N):
            Fu[t, p] = Alpha[t, p] + Beta[t, p]
    for t in range(L-1):
        for pi in range(N):
            for pj in range(N):
                Fp[t, pi, pj] = Alpha[t, pi] + Cp[pi, pj] + Cu[pj] + Beta[t+1, pj]
                
    # identify the best path/walk: adapted from the IJCAI01 paper
    y_best = np.ones(L, dtype=np.int) * (-1)
    y_best[0] = ps
    maxix = np.argmax(Fp[0, ps, :])  # the start POI is specified
    y_best[1] = maxix % N
    for t in range(2, L): 
        y_best[t] = np.argmax(Fp[t-1, y_best[t-1], :])
        
    Q = []  # priority queue (min-heap)
    maxIter = np.power(N,L-1) - np.prod([N-kx for kx in range(1,L)]) + 1 #gauranteed to find a path in maxIter iterations
    if debug == True: maxIter = np.min([maxIter, 200]); print('#iterations:', maxIter) 
        
    # heap item for the best path/walk
    priority, partition_index, exclude_set = -np.max(Fu[L-1, :]), None, set()  # -1 * score as priority
    hq.heappush(Q, HeapItem(priority, (y_best, partition_index, exclude_set)))
    
    histories = set()
        
    k = 0
    while len(Q) > 0 and k < maxIter:
        #print('------------------\n', Q, '\n------------------')
        hitem = hq.heappop(Q)
        k_priority, (k_best, k_partition_index, k_exclude_set) = hitem.priority, hitem.task
        k += 1
        
        histories.add(''.join([str(x) + ',' for x in k_best]))
        #print(k, len(histories))
        
        #print('pop:', k_priority, k_best, k_partition_index, k_exclude_set)
        if debug == True: 
            print(k_best, -k_priority)
        else:
            if len(set(k_best)) == L: return k_best
            
        
        # identify the (k+1)-th best path/walk given the 1st, 2nd, ..., k-th best: adapted from the IJCAI01 paper
        partition_index_start = 1
        if k_partition_index is not None:
            assert(k_partition_index > 0)
            assert(k_partition_index < L)
            partition_index_start = k_partition_index
            
        for parix in range(partition_index_start, L):    
            new_exclude_set = set({k_best[parix]})
            if parix == partition_index_start:
                new_exclude_set = new_exclude_set | k_exclude_set
            
            new_best = np.ones(L, dtype=np.int) * (-1)
            for pk in range(parix):
                new_best[pk] = k_best[pk]
            
            candidate_points = [p for p in range(N) if p not in new_exclude_set]
            if len(candidate_points) == 0: continue
            candidate_maxix = np.argmax([Fp[parix-1, k_best[parix-1], p] for p in candidate_points])
            new_best[parix] = candidate_points[candidate_maxix]
            
            for pk in range(parix+1, L):
                new_best[pk] = np.argmax([Fp[pk-1, new_best[pk-1], p] for p in range(N)])
            
            new_priority = Fp[parix-1, k_best[parix-1], new_best[parix]]
            if k_partition_index is not None:
                new_priority += (-k_priority) - Fp[parix-1, k_best[parix-1], k_best[parix]]
            new_priority *= -1.0  # NOTE: -np.inf - np.inf + np.inf = nan
            
            #if debug == True and np.isnan(new_priority):
            #    print(Fp[parix-1,k_best[parix-1],new_best[parix]], (-k_priority), \
            #          Fp[parix-1,k_best[parix-1],k_best[parix]])
            #    print(Fp[parix-1,k_best[parix-1],new_best[parix]] - k_priority - \
            #          Fp[parix-1,k_best[parix-1],k_best[parix]])
                
            #print(' '*3, 'push:', new_priority, new_best, parix, new_exclude_set)
            
            hq.heappush(Q, HeapItem(new_priority, (new_best, parix, new_exclude_set)))
            #print('------------------\n', Q, '\n------------------')
    if debug == True: print('#iterations: %d, #distinct_trajectories: %d' % (k, len(histories)))

In [ ]:
#do_inference_listViterbi(ps0, L0, N0, w_u, w_p, f_u, f_p, debug=True)

In [ ]:
#print(do_inference_listViterbi(ps0, L0, N0, w_u, w_p, f_u, f_p))

# MEMM with first order MC

Compute node features (singleton).

In [ ]:
def calc_node_features(startPOI, nPOI, poi_ix, poi_info, poi_clusters, cats, clusters):
    """
    Generate feature vectors for all POIs given query (startPOI, nPOI)
    """
    assert(isinstance(cats, list))
    assert(isinstance(clusters, list))
    
    columns = DF_COLUMNS[3:]
    poi_distmat = POI_DISTMAT
    query_id_dict = QUERY_ID_DICT
    key = (p0, trajLen) = (startPOI, nPOI)
    assert(key in query_id_dict)
    assert(p0 in poi_info.index)
    
    # DEBUG: use uniform node features
    nrows = len(poi_ix)
    ncols = len(columns) + len(cats) + len(clusters) - 2
    #return np.ones((nrows, ncols), dtype=np.float)
    #return np.zeros((nrows, ncols), dtype=np.float)
    
    poi_list = poi_ix
    df_ = pd.DataFrame(index=np.arange(len(poi_list)), columns=columns)
        
    for i in range(df_.index.shape[0]):
        poi = poi_list[i]
        lon, lat = poi_info.loc[poi, 'poiLon'], poi_info.loc[poi, 'poiLat']
        pop, nvisit = poi_info.loc[poi, 'popularity'], poi_info.loc[poi, 'nVisit']
        cat, cluster = poi_info.loc[poi, 'poiCat'], poi_clusters.loc[poi, 'clusterID']
        duration = poi_info.loc[poi, 'avgDuration']
        idx = df_.index[i]
        df_.set_value(idx, 'category', tuple((cat == np.array(cats)).astype(np.int) * 2 - 1))
        df_.set_value(idx, 'neighbourhood', tuple((cluster == np.array(clusters)).astype(np.int) * 2 - 1))
        df_.loc[idx, 'popularity'] = LOG_SMALL if pop < 1 else np.log10(pop)
        df_.loc[idx, 'nVisit'] = LOG_SMALL if nvisit < 1 else np.log10(nvisit)
        df_.loc[idx, 'avgDuration'] = LOG_SMALL if duration < 1 else np.log10(duration)
        df_.loc[idx, 'trajLen'] = trajLen
        df_.loc[idx, 'sameCatStart'] = 1 if cat == poi_all.loc[p0, 'poiCat'] else -1
        df_.loc[idx, 'distStart'] = poi_distmat.loc[poi, p0]
        df_.loc[idx, 'diffPopStart'] = pop - poi_info.loc[p0, 'popularity']
        df_.loc[idx, 'diffNVisitStart'] = nvisit - poi_info.loc[p0, 'nVisit']
        df_.loc[idx, 'diffDurationStart'] = duration - poi_info.loc[p0, 'avgDuration']
        df_.loc[idx, 'sameNeighbourhoodStart'] = 1 if cluster == poi_clusters.loc[p0, 'clusterID'] else -1
    
    # features other than category and neighbourhood
    X = df_[list(set(df_.columns) - {'category', 'neighbourhood'})].values  
    
    # boolean features: category (+1, -1)
    cat_features = np.vstack([list(df_.loc[x, 'category']) for x in df_.index])
    
    # boolean features: neighbourhood (+1, -1)
    neigh_features = np.vstack([list(df_.loc[x, 'neighbourhood']) for x in df_.index])
    
    return np.hstack([X, cat_features, neigh_features]).astype(np.float)

Compute edge features (transiton / pairwise).

In [ ]:
def calc_edge_features(trajid_list, poi_ix, traj_dict, poi_info):    
    feature_names = ['poiCat', 'popularity', 'nVisit', 'avgDuration', 'clusterID']
    n_features = len(feature_names)
    
    # DEBUG: use uniform edge features
    #return np.ones((len(poi_ix), len(poi_ix), n_features), dtype=np.float)
    #return np.zeros((len(poi_ix), len(poi_ix), n_features), dtype=np.float)
    
    transmat_cat                        = gen_transmat_cat(trajid_list, traj_dict, poi_info)
    transmat_pop,      logbins_pop      = gen_transmat_pop(trajid_list, traj_dict, poi_info)
    transmat_visit,    logbins_visit    = gen_transmat_visit(trajid_list, traj_dict, poi_info)
    transmat_duration, logbins_duration = gen_transmat_duration(trajid_list, traj_dict, poi_info)
    transmat_neighbor, poi_clusters     = gen_transmat_neighbor(trajid_list, traj_dict, poi_info)
    
    poi_features = pd.DataFrame(data=np.zeros((len(poi_ix), len(feature_names))), \
                                columns=feature_names, index=poi_ix)
    poi_features.index.name = 'poiID'
    poi_features['poiCat'] = poi_info.loc[poi_ix, 'poiCat']
    poi_features['popularity'] = np.digitize(poi_info.loc[poi_ix, 'popularity'], logbins_pop)
    poi_features['nVisit'] = np.digitize(poi_info.loc[poi_ix, 'nVisit'], logbins_visit)
    poi_features['avgDuration'] = np.digitize(poi_info.loc[poi_ix, 'avgDuration'], logbins_duration)
    poi_features['clusterID'] = poi_clusters.loc[poi_ix, 'clusterID']
    
    edge_features = np.zeros((len(poi_ix), len(poi_ix), n_features), dtype=np.float64)
    
    for j in range(len(poi_ix)): # NOTE: POI order
        pj = poi_ix[j]
        cat, pop = poi_features.loc[pj, 'poiCat'], poi_features.loc[pj, 'popularity']
        visit, cluster = poi_features.loc[pj, 'nVisit'], poi_features.loc[pj, 'clusterID']
        duration = poi_features.loc[pj, 'avgDuration']
        
        for k in range(len(poi_ix)): # NOTE: POI order
            pk = poi_ix[k]
            edge_features[j, k, :] = np.log10( np.array(
                    [transmat_cat.loc[cat, poi_features.loc[pk, 'poiCat']], \
                     transmat_pop.loc[pop, poi_features.loc[pk, 'popularity']], \
                     transmat_visit.loc[visit, poi_features.loc[pk, 'nVisit']], \
                     transmat_duration.loc[duration, poi_features.loc[pk, 'avgDuration']], \
                     transmat_neighbor.loc[cluster, poi_features.loc[pk, 'clusterID']]] ) )
    return edge_features

Cost function for MEMM with first order MC.

In [ ]:
%%cython
import numpy as np
from scipy.misc import logsumexp
cimport numpy as np

cpdef cost_MEMM_1MC(w, X_node, X_edge, list Y, float C, long M):
    """
    w - parameter vector
    X_node - feature matrix of POIs for all training examples, (N x M) x n_node_features
    X_edge - transition feature matrix of POIs, M x M x n_edge_features
    Y - labels/trajectories for all training examples
    C - regularisation constant
    M - total number of POIs
    """
    #print('entering cost_MEMM')
    assert(C > 0)
    assert(M > 0)
    cdef long N, D, i, j, pj, pk, pl
    N = int(np.shape(X_node)[0]/M)
    D = np.shape(X_node)[1] * 2 + np.shape(X_edge)[2]
    assert(D == np.shape(w)[0])
    assert(N == len(Y))
    
    cdef double result
    result = 0.0
    for i in range(N):
        for j in range(1, np.shape(Y[i])[0]):
            pj = Y[i][j-1]  # index of feature vector for POI p_{j-1}
            pk = Y[i][j]
            result -= np.dot(w, np.hstack([X_node[i*M + pj], X_edge[pj, pk], X_node[i*M + pk]]))
            result += logsumexp([np.dot(w, np.hstack([X_node[i*M + pj], X_edge[pj, pl], X_node[i*M + pl]])) \
                                 for pl in range(M)])
    #print('exit cost_MEMM')
    return 0.5 * np.dot(w, w) + result * C / N  # note the normalisation

Gradient of cost function for MEMM with first order MC.

In [ ]:
%%cython
import numpy as np
cimport numpy as np

cpdef grad_MEMM_1MC(w, X_node, X_edge, list Y, float C, long M):
    """
    w - parameter vector
    X_node - feature matrix of POIs for all training examples, (N x M) x n_node_features
    X_edge - transition feature matrix of POIs, M x M x n_edge_features
    Y - labels/trajectories for all training examples
    C - regularisation constant
    M - total number of POIs
    """
    #print('entering grad_MEMM')
    assert(C > 0)
    assert(M > 0)
    cdef long N, D, i, j, pj, pk, pl
    N = int(np.shape(X_node)[0]/M)
    D = np.shape(X_node)[1] * 2 + np.shape(X_edge)[2]
    assert(D == np.shape(w)[0])
    assert(N == len(Y))
    
    cdef float denorminator
    grad = np.zeros(D, dtype=np.float)
    for i in range(N):
        for j in range(1, np.shape(Y[i])[0]):
            pj = Y[i][j-1]  # index of feature vector for POI p_{j-1}
            pk = Y[i][j]
            grad -= np.hstack([X_node[i*M + pj], X_edge[pj, pk], X_node[i*M + pk]])
            denorminator = 0.0
            numerator = np.zeros(D, dtype=np.float)
            for pl in range(M):
                feature = np.hstack([X_node[i*M + pj], X_edge[pj, pl], X_node[i*M + pl]])
                term = np.exp(np.dot(w, feature))
                numerator = numerator + term * feature
                denorminator += term
            grad = grad + numerator / denorminator
    #print('exit grad_MEMM')
    return w + grad * C / N  # note the normalisation

Inference for MEMM with first order MC using the Viterbi algorithm.

In [ ]:
def inference_MEMM_1MC(ps, L, M, w, X_node, X_edge):
    assert(L > 1)
    assert(M >= L)
    assert(ps >= 0)
    assert(ps < M)
    assert(w.shape[0] == X_node.shape[1]*2 + X_edge.shape[2])
    
    A = np.zeros((L-1, M), dtype=np.float)     # scores matrix
    B = np.ones((L-1, M), dtype=np.int) * (-1) # backtracking pointers
    
    for p in range(M): # ps--p
        A[0, p] = np.dot(w, np.hstack([X_node[ps, :], X_edge[ps, p], X_node[p, :]])) if ps != p else -np.inf
        B[0, p] = ps

    for t in range(0, L-2): # ps~~p1--p
        for p in range(M):
            scores = [np.dot(w, np.hstack([X_node[p1,:], X_edge[p1,p], X_node[p,:]])) if p1 not in {p,ps} else -np.inf \
                      for p1 in range(M)]
            maxix = np.argmax(scores)
            A[t+1, p] = scores[maxix]
            B[t+1, p] = maxix

    y_hat = [np.argmax(A[L-2, :])]
    p, t = y_hat[-1], L-2
    while t >= 0:
        y_hat.append(B[t, p])
        p, t = y_hat[-1], t-1
    y_hat.reverse()

    return np.asarray(y_hat)

In [ ]:
if run_memm1m == True:
    recdict_memm1m = dict()
    cnt = 1
    keys = sorted(TRAJ_GROUP_DICT.keys())
    for i in range(len(keys)):
        t0 = time.time()
        ps, L = keys[i]
        assert(L > 1) # length > 1
        
        # training set
        trajid_set_train = set(trajid_set_all) - TRAJ_GROUP_DICT[keys[i]]
        poi_info = calc_poi_info(list(trajid_set_train), traj_all, poi_all)
        assert(L <= poi_info.shape[0])
                
        # build POI_ID <--> POI__INDEX mapping for POIs used to train CRF
        # which means only POIs in traj such that len(traj) >= 2 are included
        poi_set = set()
        for x in trajid_set_train:
            if len(traj_dict[x]) >= 2:
                poi_set = poi_set | set(traj_dict[x])
        poi_ix = sorted(poi_set)
        poi_id_dict, poi_id_rdict = dict(), dict()
        for idx, poi in enumerate(poi_ix):
            poi_id_dict[poi] = idx
            poi_id_rdict[idx] = poi
            
        # start should be in training set
        if ps not in poi_set: continue
            
        print('query #%d: start=%d, length=%d ->' % (cnt, ps, L)); cnt += 1; sys.stdout.flush()
        
        # generate training data
        train_traj_list = [traj_dict[x] for x in trajid_set_train if len(traj_dict[x]) >= 2]
        node_features_list = Parallel(n_jobs=N_JOBS)\
                             (delayed(calc_node_features)\
                              (tr[0], len(tr), poi_ix, poi_info, poi_clusters=POI_CLUSTERS, \
                               cats=POI_CAT_LIST, clusters=POI_CLUSTER_LIST) for tr in train_traj_list)
        edge_features = calc_edge_features(list(trajid_set_train), poi_ix, traj_dict, poi_info)       
        assert(len(train_traj_list) == len(node_features_list))
        X_node = np.vstack(node_features_list)
        X_edge = edge_features
        y_train = [np.array([poi_id_dict[x] for x in tr]) for tr in train_traj_list]
        
        # feature scaling
        if MIN_MAX_SCALE == True:
            scaler = MinMaxScaler(feature_range=(-1,1), copy=False)
        else:
            scaler = StandardScaler(copy=False)
        X_node = scaler.fit_transform(X_node)
                        
        # train and test
        w = np.random.rand(X_node.shape[1] * 2 + X_edge.shape[2])  # initial guess
        opt_method = 'BFGS' # 'Newton-CG'
        opt = minimize(cost_MEMM_1MC, w, args=(X_node, X_edge, y_train, LOGIT_C, len(poi_ix)), \
                       method=opt_method, jac=grad_MEMM_1MC, options={'disp': True})
        if opt.success == True:
            w = opt.x
            
            # generate test data
            X_node_test = calc_node_features(ps, L, poi_ix, poi_info, poi_clusters=POI_CLUSTERS, \
                                             cats=POI_CAT_LIST, clusters=POI_CLUSTER_LIST)
            X_node_test = scaler.transform(X_node_test)
        
            # prediction (Viterbi)
            y_hat = inference_MEMM_1MC(poi_id_dict[ps], L, len(poi_ix), w, X_node_test, X_edge)
            
            recdict_memm1m[(ps, L)] = [poi_id_rdict[x] for x in y_hat]
            print(' '*10, recdict_memm1m[(ps, L)])
            
        else:
            sys.stderr.write('Optimisation failed\n')
            continue
        if cnt == 3: break

In [ ]:
if run_memm1m == True:
    F1_memm1m = []; pF1_memm1m = []; Tau_memm1m = []
    for key in sorted(recdict_memm1m.keys()):
        F1, pF1, tau = evaluate(recdict_memm1m[key], TRAJ_GROUP_DICT[key])
        F1_memm1m.append(F1); pF1_memm1m.append(pF1); Tau_memm1m.append(tau)
    print('MEMM_1MC: F1 (%.3f, %.3f), pairsF1 (%.3f, %.3f), Tau (%.3f, %.3f)' % \
          (np.mean(F1_memm1m), np.std(F1_memm1m)/np.sqrt(len(F1_memm1m)), \
           np.mean(pF1_memm1m), np.std(pF1_memm1m)/np.sqrt(len(pF1_memm1m)), \
           np.mean(Tau_memm1m), np.std(Tau_memm1m)/np.sqrt(len(Tau_memm1m))))